## LCEL and the Chain Interface

### What is LCEL?

The **LangChain Expression Language (LCEL)** is a declarative approach for efficiently describing and constructing chains. It simplifies the process of chaining components by focusing on _what_ needs to happen rather than _how_ it should be implemented.

---

### When to Use LCEL vs. the Chain Interface

While simple applications may only require a single LLM, more complex workflows often involve integrating LLMs with other components. LangChain provides two key frameworks for building such workflows:

1. **Chain Interface**: The traditional, procedural method for constructing chains.
2. **LCEL**: A modern, declarative alternative designed for simplicity and flexibility.

For new applications, **LCEL** is the recommended choice due to its optimized execution and ease of use. However, the **Chain interface** can still be incorporated within LCEL, enabling a hybrid approach that leverages the strengths of both frameworks.

# Advantages of Using LCEL

The **LangChain Expression Language (LCEL)** offers several key advantages that enhance efficiency, flexibility, and observability when building and managing chains:

---

### 1. **Asynchronous, Batch, and Streaming Support**
LCEL chains natively support:
- **Synchronous** and **Asynchronous** execution.
- **Batch processing** for handling multiple inputs simultaneously.
- **Streaming** for real-time, incremental output generation.

This versatility enables developers to:
- Start with a simple synchronous prototype.
- Seamlessly transition to an asynchronous, streaming-based interface as application demands grow.

---

### 2. **Built-in Fallback Mechanism**
LCEL makes it easy to integrate fallbacks within chains, ensuring:
- Robust error handling.
- Graceful recovery from failures without disrupting the chain's overall flow.

---

### 3. **Optimized Parallel Processing**
LCEL chains are designed for **parallel execution** of their components. This is especially beneficial for LLM-based workflows that involve:
- Lengthy API calls.
- High latency operations.

Parallelism significantly reduces processing time, making LCEL ideal for performance-critical applications.

---

### 4. **Seamless Integration with LangSmith**
LCEL automatically logs every step of chain execution in **LangSmith**, offering:
- Maximum **observability** for monitoring and troubleshooting.
- Enhanced **debuggability** for complex workflows.

This integration ensures transparency and simplifies the process of identifying and resolving issues within chains.

!pip install langchain==0.0.321

In [2]:
import os
from rich import print as pp

## Describe the chain with "LCEL".
#### The chain that connects "prompt template → model" is written as " prompt | model ".

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

model = ChatOllama(model='llama3.2:1b')

prompt = ChatPromptTemplate.from_template("Tell a joke about {topic}")
chain = prompt | model

In [4]:
pp(chain)

RunnableSequence(
    first=ChatPromptTemplate(
        input_variables=['topic'],
        input_types={},
        partial_variables={},
        messages=[
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['topic'],
                    input_types={},
                    partial_variables={},
                    template='Tell a joke about {topic}'
                ),
                additional_kwargs={}
            )
        ]
    ),
    middle=[],
    last=ChatOllama(model='llama3.2:1b')
)

## Streaming responses

In [5]:
for s in chain.stream({"topic": "Programming"}):
    print(s.content, end="", flush=True)

A programmer walked into a bar and ordered a beer. As he was sipping his drink, he heard a voice say, "Nice tie!" He looked around, but there was nobody nearby who could have said it. A few minutes later, he heard the same voice say, "Beautiful shirt!" Again, he looked around, but he couldn't find anyone who might have spoken. A few more minutes passed, and he heard the voice say, "Great haircut!" This time, he decided to investigate further. He asked the bartender, "Did you hear that voice?"

The bartender replied, "Oh, that's just the peanuts. They're complementary."

# LangChain Components and the `Runnable` Protocol

LangChain components follow the **Runnable protocol**, which defines a standardized interface for creating and executing custom chains in a consistent and reusable manner.

---

## Standard Interface

The `Runnable` protocol provides both synchronous and asynchronous methods for interacting with components:

### Synchronous Methods
- **`stream`**: Streams chunks of the response back in real-time.
- **`invoke`**: Executes the chain with a given input and returns the result.
- **`batch`**: Processes a list of inputs through the chain and returns a list of outputs.

### Asynchronous Methods
- **`astream`**: Asynchronously streams chunks of the response in real-time.
- **`ainvoke`**: Asynchronously invokes the chain with a single input.
- **`abatch`**: Asynchronously processes a batch of inputs and returns corresponding outputs.
- **`astream_log`**: Streams intermediate steps along with the final response for enhanced debugging.

---

## Input and Output Types

The types of input and output vary depending on the specific component in use. 

### Input Types
- **Prompt**: Accepts a dictionary.
- **Retriever**: Takes a single string.
- **LLM / ChatModel**: Accepts a single string, a list of messages, or a `PromptValue`.
- **Tool**: May accept a single string or a dictionary, depending on the tool's implementation.
- **OutputParser**: Works with outputs from an LLM or ChatModel.

### Output Types
- **LLM**: Produces a string.
- **ChatModel**: Outputs a chat message object.
- **Prompt**: Returns a `PromptValue`.
- **Retriever**: Provides a list of documents.
- **Tool**: Output depends on the tool's functionality.
- **OutputParser**: Output varies depending on the parser's implementation.

---

## Inspecting Input and Output Schemas

You can validate the expected input and output types of a component using its schema definitions, which follow the **Pydantic** framework:

- **`input_schema`**: Defines the schema for input types.
- **`output_schema`**: Defines the schema for output types.

These schemas ensure compatibility and provide a clear structure for integrating components into workflows.

In [22]:
pp(chain.input_schema.schema())

{
    'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
    'required': ['topic'],
    'title': 'PromptInput',
    'type': 'object'
}

In [23]:
pp(prompt.input_schema.schema())

{
    'properties': {
        'context': {'title': 'Context', 'type': 'string'},
        'question': {'title': 'Question', 'type': 'string'}
    },
    'required': ['context', 'question'],
    'title': 'PromptInput',
    'type': 'object'
}

In [25]:
model.input_schema.schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

In [26]:
chain.output_schema.schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

In [10]:
pp(chain.invoke({"topic": "Football"}))

AIMessage(
    content='Why did the football go to the doctor?\n\nBecause it was feeling a little deflated.',
    additional_kwargs={},
    response_metadata={
        'model': 'llama3.2:1b',
        'created_at': '2025-01-13T16:38:57.026434Z',
        'message': {'role': 'assistant', 'content': ''},
        'done_reason': 'stop',
        'done': True,
        'total_duration': 249027958,
        'load_duration': 28573500,
        'prompt_eval_count': 30,
        'prompt_eval_duration': 18000000,
        'eval_count': 19,
        'eval_duration': 200000000
    },
    id='run-83caea35-a628-4ad4-8417-14816cbbda90-0',
    usage_metadata={'input_tokens': 30, 'output_tokens': 19, 'total_tokens': 49}
)

In [11]:
pp(chain.batch([{"topic": "Love"}, {"topic": "Romance"}]))

[
    AIMessage(
        content='Why did Love go to therapy?\n\nBecause it was feeling a little "unhinged."',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:38:57.213477Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 178958041,
            'load_duration': 12742166,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 14000000,
            'eval_count': 19,
            'eval_duration': 151000000
        },
        id='run-ae289a2f-8dff-4a30-9807-76faeb773b30-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 19, 'total_tokens': 49}
    ),
    AIMessage(
        content='A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and 
Schrödinger\'s cat?" The librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:38:57.456737Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 421970500,
            'load_duration': 12649042,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 40000000,
            'eval_count': 54,
            'eval_duration': 368000000
        },
        id='run-c007be30-bac8-4b54-bc83-7e0c1caba124-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 54, 'total_tokens': 84}
    )
]

In [12]:
pp(chain.batch([{"topic": "Coding"}, {"topic": "Travelling"}], config={"max_concurrency": 5}))

[
    AIMessage(
        content='A programmer walks into a library and asks the librarian, "Do you have any books on computer 
science?" The librarian replies, "It\'s a bit of a \'byte\' size problem, but I think we can \'debug\' that for 
you."',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:38:57.871531Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 405698292,
            'load_duration': 10778375,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 10000000,
            'eval_count': 51,
            'eval_duration': 384000000
        },
        id='run-a53c9965-2029-493d-9703-75636a12fea5-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 51, 'total_tokens': 81}
    ),
    AIMessage(
        content='A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and 
Schrödinger\'s cat?"\n\nThe librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:38:57.904136Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 438267250,
            'load_duration': 11209792,
            'prompt_eval_count': 31,
            'prompt_eval_duration': 29000000,
            'eval_count': 54,
            'eval_duration': 396000000
        },
        id='run-65371929-9f19-4965-986b-5270024afc6d-0',
        usage_metadata={'input_tokens': 31, 'output_tokens': 54, 'total_tokens': 85}
    )
]

### Async Stream

In [13]:
async for s in chain.astream({"topic": "Satellites"}):
    print(s.content, end="", flush=True)

Why did the satellite go to therapy?

Because it had a lot of "orbital" issues and was feeling "out of balance."

### Async Invoke

In [14]:
async for s in chain.astream({"topic": "Food"}):
    print(s.content, end="", flush=True)

Why was the pizza in a bad mood?

Because it was feeling crusty.

### Async Batch

In [15]:
c =await chain.abatch([{"topic": "Food"}, {"topic": "Sweets"}])
pp(c)

[
    AIMessage(
        content='A man walks into a library and asks the librarian, "Do you have any books on Pavlov\'s dogs and 
Schrödinger\'s cat?" \n\nThe librarian replies, "It rings a bell, but I\'m not sure if it\'s here or not."',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:38:58.674882Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 411619167,
            'load_duration': 10872167,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 6000000,
            'eval_count': 55,
            'eval_duration': 393000000
        },
        id='run-afcc35c5-fbb5-40b1-8468-d651785999d4-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 55, 'total_tokens': 85}
    ),
    AIMessage(
        content='Why did the lollipop go to therapy?\n\nBecause it was feeling a little "sour" and wanted to work 
through some "candy" issues.',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:38:58.541922Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 277920458,
            'load_duration': 10285125,
            'prompt_eval_count': 31,
            'prompt_eval_duration': 26000000,
            'eval_count': 32,
            'eval_duration': 240000000
        },
        id='run-c28de374-190c-4bcf-9e7e-4cd46495ebb6-0',
        usage_metadata={'input_tokens': 31, 'output_tokens': 32, 'total_tokens': 63}
    )
]

### Async Stream with intermediate steps

Useful for displaying progress to the user, working with intermediate results, and debugging chains. You can stream all steps (default) or include or exclude steps by name, tags, or metadata.

In [27]:
# !pip install faiss-cpu tiktoken

Execute the Retriever chain and output intermediate steps using astream_log()

In [29]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings 

template = """Please answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(["Sonu is the creator of AI Anytime Youtube Channel"], embedding=OllamaEmbeddings(model="snowflake-arctic-embed:33m"))
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {"context": retriever.with_config(run_name='Docs'), "question": RunnablePassthrough()}
    | prompt 
    | model 
    | StrOutputParser()
)
pp(retrieval_chain)
async for chunk in retrieval_chain.astream_log("Who is the creator of AI Anytime?", include_names=['Docs']):
    print("-"*40)
    print(chunk)

RunnableSequence(
    first=RunnableParallel(
        steps__={
            'context': RunnableBinding(
                bound=VectorStoreRetriever(
                    tags=['FAISS', 'OllamaEmbeddings'],
                    vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12fe1dc90>,
                    search_kwargs={}
                ),
                kwargs={},
                config={'run_name': 'Docs'},
                config_factories=[]
            ),
            'question': RunnablePassthrough()
        }
    ),
    middle=[
        ChatPromptTemplate(
            input_variables=['context', 'question'],
            input_types={},
            partial_variables={},
            messages=[
                HumanMessagePromptTemplate(
                    prompt=PromptTemplate(
                        input_variables=['context', 'question'],
                        input_types={},
                        partial_variables={},
                        template='Please answer the question based only on the following 
context:\n{context}\n\nQuestion: {question}\n'
                    ),
                    additional_kwargs={}
                )
            ]
        ),
        ChatOllama(model='llama3.2:1b')
    ],
    last=StrOutputParser()
)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '9df6d72d-2382-4695-a640-184d5a48c213',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'ebdb5324-ee49-41e2-8779-2f29faa5042c',
            'metadata': {'ls_embedding_provider': 'OllamaEmbeddings',
                         'ls_retriever_name': 'vectorstore',
                         'ls_vector_store_provider': 'FAISS'},
            'name': 'Docs',
            'start_time': '2025-01-13T16:42:27.911+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'OllamaEmbeddings'],
            'type': 'retriever'}})
--------------------

## Parallel Processing

"RunnableParallel" allows each element to run in parallel.

In [18]:
from langchain.schema.runnable import RunnableParallel
chain1 = ChatPromptTemplate.from_template("Tell a joke about {topic}") | model
chain2 = ChatPromptTemplate.from_template("Write a short (2 line) poem about {topic}") | model
combined = RunnableParallel(joke=chain1, poem=chain2)

In [19]:
%%time

pp(chain1.batch([{"topic": "AI"}, {"topic": "Math"}]))

[
    AIMessage(
        content='A robot walked into a bar and ordered a beer. As he was sipping his drink, he said to the 
bartender, "I\'m feeling a little glitchy today." The bartender replied, "Don\'t worry, it\'s just a software 
bug."',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:39:01.727207Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 479388458,
            'load_duration': 45991458,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 10000000,
            'eval_count': 52,
            'eval_duration': 422000000
        },
        id='run-5266cb51-3c15-47af-bd4d-3b020d6d36ad-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 52, 'total_tokens': 82}
    ),
    AIMessage(
        content='A mathematician walks into a library and asks the librarian, "Do you have any books on Pavlov\'s 
dogs and Schrödinger\'s cat?" The librarian replies, "It rings a bell, but I\'m not sure if it\'s here or not."',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:39:01.759563Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 511687833,
            'load_duration': 46054375,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 34000000,
            'eval_count': 55,
            'eval_duration': 430000000
        },
        id='run-709f065c-4f43-4b22-b825-96060d261e8c-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 55, 'total_tokens': 85}
    )
]

CPU times: user 17.8 ms, sys: 3.01 ms, total: 20.9 ms
Wall time: 519 ms


In [20]:
%%time

pp(chain2.batch([{"topic": "Science"}, {"topic": "Mango"}]))

[
    AIMessage(
        content='Here is a short poem about science:\n\n"Curiosity drives us to explore and learn,\nUnlocking 
secrets, one discovery at a time."',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:39:02.018836Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 248072000,
            'load_duration': 11003958,
            'prompt_eval_count': 35,
            'prompt_eval_duration': 10000000,
            'eval_count': 29,
            'eval_duration': 226000000
        },
        id='run-d1968b67-d27a-4c9d-a978-0bf5052034fb-0',
        usage_metadata={'input_tokens': 35, 'output_tokens': 29, 'total_tokens': 64}
    ),
    AIMessage(
        content="Here is a short poem about mango:\n\nSweet and juicy, ripe and fine,\nMango's flavor is truly 
divine.",
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:39:02.005809Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 235011666,
            'load_duration': 11094250,
            'prompt_eval_count': 35,
            'prompt_eval_duration': 35000000,
            'eval_count': 25,
            'eval_duration': 188000000
        },
        id='run-ea58a397-0db0-47f1-ad16-ab1425f7e119-0',
        usage_metadata={'input_tokens': 35, 'output_tokens': 25, 'total_tokens': 60}
    )
]

CPU times: user 11.9 ms, sys: 2.03 ms, total: 13.9 ms
Wall time: 254 ms


In [21]:
%%time

# combined
pp(combined.batch([{"topic": "AI"}, {"topic": "Mountains"}]))

[
    {
        'joke': AIMessage(
            content='Why did the AI program go to therapy?\n\nBecause it was feeling a little " glitchy" and had 
trouble processing its emotions.',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:1b',
                'created_at': '2025-01-13T16:39:02.351122Z',
                'message': {'role': 'assistant', 'content': ''},
                'done_reason': 'stop',
                'done': True,
                'total_duration': 322975417,
                'load_duration': 14902458,
                'prompt_eval_count': 30,
                'prompt_eval_duration': 7000000,
                'eval_count': 27,
                'eval_duration': 300000000
            },
            id='run-e1f34da2-9fbe-42d2-bdf6-fc644f2e4aac-0',
            usage_metadata={'input_tokens': 30, 'output_tokens': 27, 'total_tokens': 57}
        ),
        'poem': AIMessage(
            content="Here's a short poem about AI:\n\nRise of the machines, a new mind\nConsciousness born, with 
code entwined.",
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:1b',
                'created_at': '2025-01-13T16:39:02.396088Z',
                'message': {'role': 'assistant', 'content': ''},
                'done_reason': 'stop',
                'done': True,
                'total_duration': 366919583,
                'load_duration': 14686625,
                'prompt_eval_count': 35,
                'prompt_eval_duration': 32000000,
                'eval_count': 30,
                'eval_duration': 318000000
            },
            id='run-22b7371d-cef4-486f-b04a-e31c2f799686-0',
            usage_metadata={'input_tokens': 35, 'output_tokens': 30, 'total_tokens': 65}
        )
    },
    {
        'joke': AIMessage(
            content='A man went to the doctor and said, "Doc, I\'ve been feeling really off lately. I\'ve got a lot
of altitude and it\'s been making my life feel like it\'s on the slope." The doctor replied, "Well, you\'re not 
just mountain-high, you\'re just in need of a boost!"',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:1b',
                'created_at': '2025-01-13T16:39:02.624802Z',
                'message': {'role': 'assistant', 'content': ''},
                'done_reason': 'stop',
                'done': True,
                'total_duration': 595130500,
                'load_duration': 13997875,
                'prompt_eval_count': 30,
                'prompt_eval_duration': 32000000,
                'eval_count': 66,
                'eval_duration': 546000000
            },
            id='run-abe43a2c-5c20-4531-b025-efd0ffb1ac34-0',
            usage_metadata={'input_tokens': 30, 'output_tokens': 66, 'total_tokens': 96}
        ),
        'poem': AIMessage(
            content="Here is a short two-line poem about mountains:\n\nGranite peaks touch the sky so 
high,\nNature's beauty, touching the eye.",
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:1b',
                'created_at': '2025-01-13T16:39:02.380606Z',
                'message': {'role': 'assistant', 'content': ''},
                'done_reason': 'stop',
                'done': True,
                'total_duration': 350821708,
                'load_duration': 14642333,
                'prompt_eval_count': 35,
                'prompt_eval_duration': 33000000,
                'eval_count': 28,
                'eval_duration': 301000000
            },
            id='run-ca552c38-9c62-486f-b067-d2a638e95fb3-0',
            usage_metadata={'input_tokens': 35, 'output_tokens': 28, 'total_tokens': 63}
        )
    }
]

CPU times: user 29 ms, sys: 4.66 ms, total: 33.6 ms
Wall time: 611 ms
